In [ ]:
!pip install vaderSentiment

In [ ]:
!pip install gradio

In [ ]:
#Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gradio as gr
import matplotlib.pyplot as plt

In [ ]:
# Loading and merging dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
df1 = pd.read_csv('Combined_News_DJIA.csv')
df2 = pd.read_csv('upload_DJIA_table.csv')

In [ ]:
df2.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688


In [ ]:
df1.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."


In [ ]:
merged = df1.merge(df2, how='inner', on='Date')
merged.head(3)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727


In [ ]:
# Data cleaning and preprocessing

In [ ]:
headlines = []
for row in range(0,len(merged.index)):
    headlines.append(' '.join(str(x) for x in merged.iloc[row,0:25]))
headlines[1]

'2008-08-11 1 b\'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?\' b\'Bush puts foot down on Georgian conflict\' b"Jewish Georgian minister: Thanks to Israeli training, we\'re fending off Russia " b\'Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired\' b"Olympic opening ceremony fireworks \'faked\'" b\'What were the Mossad with fraudulent New Zealand Passports doing in Iraq?\' b\'Russia angered by Israeli military sale to Georgia\' b\'An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people\' b\'Welcome To World War IV! Now In High Definition!\' b"Georgia\'s move, a mistake of monumental proportions " b\'Russia presses deeper into Georgia; U.S. says regime change is goal\' b\'Abhinav Bindra wins first ever Individual Olympic Gold Medal for India\' b\' U.S. ship heads for Arctic to define territory\' b\'Drivers in a Jerusalem taxi station threat

In [ ]:
clean_headlines = []
for i in range(0, len(headlines)):
  clean_headlines.append(re.sub("b[(')]",'',headlines[i]))
  clean_headlines[i] = re.sub('b[(")]','',clean_headlines[i])
  clean_headlines[i] = re.sub("\'",'',clean_headlines[i])

In [ ]:
merged['Combined_News'] = clean_headlines

In [ ]:
# Performing sentiment analysis

In [ ]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

In [ ]:
merged['Subjectivity'] = merged['Combined_News'].apply(getSubjectivity)
merged['Polarity'] = merged['Combined_News'].apply(getPolarity)

In [ ]:
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment

In [ ]:
compound = []
pos = []
neg = []
neu = []
SIA = 0

for i in range(0, len(merged['Combined_News'])):
  SIA = getSIA(merged['Combined_News'][i])
  compound.append(SIA['compound'])
  pos.append(SIA['pos'])
  neg.append(SIA['neg'])
  neu.append(SIA['neu'])


In [ ]:
merged['Compound'] = compound
merged['Positive'] = pos
merged['Negative'] = neg
merged['Neutral'] = neu

In [ ]:
merged.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Close,Volume,Adj Close,Combined_News,Subjectivity,Polarity,Compound,Positive,Negative,Neutral
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,11734.320312,212830000,11734.320312,2008-08-08 0 Georgia downs two Russian warplan...,0.269961,-0.046731,-0.9974,0.043,0.213,0.744
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,11782.349609,183190000,11782.349609,2008-08-11 1 Why wont America and Nato help us...,0.367851,0.100954,-0.9849,0.084,0.191,0.725
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,11642.469727,173590000,11642.469727,2008-08-12 0 Remember that adorable 9-year-old...,0.539786,-0.052157,-0.9700,0.058,0.131,0.811
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,11532.959961,182550000,11532.959961,2008-08-13 0 U.S. refuses Israel weapons to a...,0.379552,0.017951,-0.9730,0.071,0.143,0.786
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,11615.929688,159790000,11615.929688,2008-08-14 1 All the experts admit that we sho...,0.369669,0.039728,-0.9881,0.083,0.184,0.733


In [ ]:
# Selecting features for model training

In [ ]:
keep_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Subjectivity', 'Polarity', 'Compound', 'Positive', 'Negative', 'Neutral', 'Label']
df = merged[keep_columns]
df.head()

,Open,High,Low,Close,Volume,Adj Close,Subjectivity,Polarity,Compound,Positive,Negative,Neutral,Label
0,11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,0.269961,-0.046731,-0.9974,0.043,0.213,0.744,0
1,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,0.367851,0.100954,-0.9849,0.084,0.191,0.725,1
2,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,0.539786,-0.052157,-0.9700,0.058,0.131,0.811,0
3,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,0.379552,0.017951,-0.9730,0.071,0.143,0.786,0
4,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,0.369669,0.039728,-0.9881,0.083,0.184,0.733,1


In [ ]:
# Train test split and model training

In [ ]:
X = df
X.drop(['Label'], axis=1)

y = df['Label']

In [ ]:
X_train,  X_test, y_train, y_test =  train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
model = LinearDiscriminantAnalysis().fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# Classification report

In [ ]:
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90       193
           1       0.86      0.97      0.91       205

    accuracy                           0.90       398
   macro avg       0.91      0.90      0.90       398
weighted avg       0.91      0.90      0.90       398



In [ ]:
def plots(attribute='Polarity'):
  df_copy = df.copy()
  values = df_copy[attribute]
  plot = plt.scatter(x=values, y=df_copy['Label'])
  plt.ylabel('Label')
  return plt.gcf()

In [ ]:
plots('Positive')

In [ ]:
iface = gr.Interface(
    plots,
    [
        gr.inputs.Dropdown(['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'Subjectivity', 'Polarity', 'Compound', 'Positive', 'Negative', 'Neutral', 'Label']),
    ],
    "plot",
    description = 'Select the attribute related to the news and stock and get plot w.r.t Label(1 shows increase in stock price and 0 shows decrease in stock price)',
)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://31894.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f0643e034d0>,
 'http://127.0.0.1:7868/',
 'https://31894.gradio.app')